# Kaggle: CIBMTR - Equity in post-HCT Survival Predictions

Final Training Notebook of Kaggle CIBMTR competition

Some of the code where from Chris Deotte Baseline Notebook:
https://www.kaggle.com/code/cdeotte/gpu-lightgbm-baseline-cv-681-lb-685#XGBoost-with-Survival:Cox

## Load Train and Test

In [ ]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from metric import score
from sklearn.model_selection import KFold
from scipy.stats import gamma
from lifelines import KaplanMeierFitter, NelsonAalenFitter
from scipy.stats import rankdata 
import joblib
import pickle
import random
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import warnings

path = "/kaggle/input/"
path = ""

# Load dictionary
with open("category_mappings.pkl", "rb") as file:
    category_mappings = pickle.load(file)
    
test = pd.read_csv("equity-post-HCT-survival-predictions/test.csv")
print("Test shape:", test.shape )

train = pd.read_csv("equity-post-HCT-survival-predictions/train.csv")
print("Train shape:",train.shape)

train.head()

# Suppress only FutureWarning messages
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

# Feature Engineering

In [ ]:
dri_mapping = {
    "Very high": 3.2,
    "High": 2.2,
    "High - TED AML case <missing cytogenetics": 2.8,
    "Intermediate": 1.4,
    "Intermediate - TED AML case <missing cytogenetics": 1.7,
    "Low": 0.9,
    "TBD cytogenetics": 0.8,
    "N/A - non-malignant indication": 0.4,
    "N/A - disease not classifiable": 0.3,
    "N/A - pediatric": 0.5,
    "Missing disease status": 0
}

sex_match_mapping = {
    'M-F': 1, 
    'F-F': 0.5,
    'F-M': 0.5,
    'M-M':1
}

cyto_mapping = {
    "Favorable": 4,
    "Intermediate": 3,
    "Poor": 2,
    "Normal": 1,
    "Other": 0,
    "Not tested": -1,
    "TBD": -1
}

cyto_detail_mapping = {
    "Favorable": 4,
    "Intermediate": 3,
    "Poor": 2,
    "Not tested": -1,
    "TBD": -1
}

tbi_mapping = {
    "No TBI": 0,
    "TBI + Cy +- Other": 0.3,
    "TBI +- Other, <=cGy": 0.4,  # Lower dose, lower impact
    "TBI +- Other, >cGy": 0.5,   # Higher dose, higher impact
    "TBI +- Other, -cGy, single": 0.4,  # Similar to >cGy
    "TBI +- Other, -cGy, fractionated": 0.2,  # Lower risk due to fractionation
    "TBI +- Other, -cGy, unknown dose": 0.5,  # Slightly increased risk
    "TBI +- Other, unknown dose": 0.6  # Highest risk (uncertain)
}


conditioning_mapping = {
    "MAC": 4,  # Most intense
    "RIC": 3,  # Intermediate intensity
    "NMA": 2,  # Least intense
    "TBD": 0.8,  # Unknown
    "No drugs reported": 0,  # No conditioning
    "N/A, F(pre-TED) not submitted": 0.5,  # Missing data
}

disease_mapping = {
    "ALL": "1",  # Acute Lymphoblastic Leukemia
    "AML": "1",  # Acute Myeloid Leukemia
    "MDS": "1.5",  # Myelodysplastic Syndrome
    "CML": "1.5",  # Chronic Myeloid Leukemia
    "Other acute leukemia": "1", # Other acute leukemia types
    "MPN": "2",  # Myeloproliferative Neoplasms
    "HD": "2.5",  # Hodgkin Disease (Lymphoma)
    "NHL": "2.5",  # Non-Hodgkin Lymphoma
    "PCD": "2",  # Plasma Cell Disorders (e.g., multiple myeloma)
    "SAA": "3", # Severe Aplastic Anemia
    "AI": "3",  # Autoimmune Disorders
    "HIS": "3", # Hemophagocytic Immune Syndromes
    "IMD": "4.5",  # Inherited Metabolic Disorders
    "IPA": "4.5",  # Inherited Primary Immunodeficiency
    "IEA": "4.5",  # Inherited Erythrocyte Abnormalities
    "Solid tumor": "4",  # Non-blood cancers requiring HCT
    "IIS": "4.5"  # Inherited Immune System Disorders
}

disease_num_mapping = {
    "ALL": 0.7,
    "AML": 0.7,
    "MDS": 0.75,
    "CML": 1,
    "Other leukemia": 0.3,
    "Other acute leukemia": 0.6,
    "MPN": 0.7,
    "HD": 0.3,
    "NHL": 0.6,
    "PCD": 0.3,
    "SAA": 0.4,
    "AI": 0.8,
    "HIS": 0.1,
    "IMD": 0.2,
    "IPA": 0.85,
    "IEA": 0.2,
    "Solid tumor": 0.2,
    "IIS": 0
}

tce_mapping = {
    "P/P": 4,  # Best match
    "G/G": 3,
    "H/H": 3,
    "G/B": 2,
    "H/B": 2,
    "P/H": 1,
    "P/B": 0.5,
    "P/G": 0   # Worst match
}

tce_match_mapping = {
    "Permissive": 0.5,            
    "GvH non-permissive": 1,    
    "Fully matched": 0,          
    "HvG non-permissive": 1       
}

tce_div_match_mapping = {
    "Permissive mismatched": 0.7,            
    "GvH non-permissive": 1,               
    "HvG non-permissive":1,     
    "Bi-directional non-permissive":2      
}

mel_mapping = {
    "N/A, Mel not given": 0, 
    np.nan: 0,
    "MEL": 1
}

graft_prod_mapping = {
    "Peripheral blood_PB": 1,
    "Bone marrow_BM": 0,          
    "Peripheral blood_BM":0.5,      
    "Bone marrow_PB": 0.5          
}

pulm_mapping = {
    "No": 0,                
    "Yes": 1,           
    "Not done":0.5,     
    np.nan:0.5
}

gvhd_proph_mapping = {
    # **CNI-Based (Tacrolimus - FK)**
    "FK+- others(not MMF,MTX)": "0",
    "FK+ MMF +- others": "0",
    "FK+ MTX +- others(not MMF)": "1",
    "FK alone": "0",

    # **CNI-Based (Cyclosporine - CSA)**
    "CSA alone": "0",
    "CSA +- others(not FK,MMF,MTX)": "0",
    "CSA + MMF +- others(not FK)": "0",
    "CSA + MTX +- others(not MMF,FK)": "0",

    # **Post-Transplant Cyclophosphamide (PTCy)**
    "Cyclophosphamide alone": "1",
    "Cyclophosphamide +- others": "1",

    # **T-Cell Depletion Strategies**
    "TDEPLETION alone": "2",
    "TDEPLETION +- other": "2",
    "CDselect alone": "2",
    "CDselect +- other": "2",

    # **No Prophylaxis / Minimal Immunosuppression**
    "No GvHD Prophylaxis": "3",
    "Parent Q = yes, but no agent": "3",

    # **Other / Unclassified**
    "Other GVHD Prophylaxis": "4"
}

mrd_mapping = {
    'Positive':1,
    'Negative':0,
    np.nan: 0.5
}
# Define CMV risk mapping
cmv_mapping = {
    "+/+": 1,
    "-/+": 0.5, 
    "+/-": 0.5,
    "-/-": 1  
}

donor_related_mapping = {
    'Unrelated': 1,
    'Related': 0,
    'Multiple donor (non-UCB)': 0.8,
    np.nan: 0.5,
    
}


In [ ]:

train["tce_match_numeric"] = train["tce_match"].map(tce_match_mapping)
train['prim_disease_hct_encoded'] = train['prim_disease_hct'].map(disease_mapping)
train['prim_disease_hct_num_encoded'] = train['prim_disease_hct'].map(disease_num_mapping)
train['conditioning_intensity_encoded'] = train['conditioning_intensity'].map(conditioning_mapping)
train['cyto_score_encoded'] = train['cyto_score'].map(cyto_mapping)

train['dri_score_encoded'] = train['dri_score'].map(dri_mapping)
train['graft_type_prod_type_encoded'] = (train["graft_type"] + "_" + train["prod_type"]).map(graft_prod_mapping)

train['dri_score_comorbidity_score'] = train['dri_score_encoded'] * train["comorbidity_score"]
train['cyto_score_comorbidity_score'] = train['cyto_score_encoded'] * train["comorbidity_score"]


hla_match_cols = ["hla_match_a_high", "hla_match_b_high", "hla_match_c_high",
                  "hla_match_drb1_high", "hla_match_dqb1_high"]
train["hla_total_match"] = train[hla_match_cols].sum(axis=1)

hla_mismatch_cols = ["hla_match_a_low", "hla_match_b_low", "hla_match_c_low",
                     "hla_match_drb1_low", "hla_match_dqb1_low"]
train["hla_total_mismatch"] = train[hla_mismatch_cols].sum(axis=1)


train['prim_disease_hct_num_encoded_cond_intensity'] = train['prim_disease_hct_num_encoded']*train["conditioning_intensity_encoded"]
train['comor_cond_intensity'] = train['comorbidity_score']*train["conditioning_intensity_encoded"]


## Feature Encoding

In [ ]:
RMV = ["ID","efs","efs_time","y"]
FEATURES = [c for c in train.columns if not c in RMV]
print(f"There are {len(FEATURES)} FEATURES: {FEATURES}")

In [ ]:
# Fill nan

CATS = []
for c in FEATURES:
    if train[c].dtype=="object":
        CATS.append(c)
        train[c] = train[c].fillna("NAN") 

print(f"In these features, there are {len(CATS)} CATEGORICAL FEATURES: {CATS}")

In [ ]:
# Label Encode The categorical Features
combined = train

print("We LABEL ENCODE the CATEGORICAL FEATURES: ",end="")

for c in FEATURES:

    # LABEL ENCODE CATEGORICAL AND CONVERT TO INT32 CATEGORY
    if c in CATS:
        print(f"{c}, ",end=" ")
        combined[c] = combined[c].map(category_mappings[c])

        combined[c] = combined[c].astype("int32")
        combined[c] = combined[c].astype("category")

    # REDUCE PRECISION OF NUMERICAL TO 32BIT TO SAVE MEMORY
    else:
        if combined[c].dtype=="float64":
            combined[c] = combined[c].astype("float32")
        if combined[c].dtype=="int64":
            combined[c] = combined[c].astype("int32")


train = combined.copy()

- More than one race 0
- Asian 1
- White 2
- American Indian or Alaska Native 3
- Native Hawaiian or other Pacific Islander 4
- Black or African-American 5

# Target Transformations

In [ ]:
class exponential:
    def __init__(self, lambda_, time_col="efs_time"):
        self.lambda_ = lambda_
        self.time_col = time_col

    def predict(self, df):
        y = self.lambda_ * np.exp(-self.lambda_ * df[self.time_col]).values
        y = y/y.max()
        
        return y

    def reset(self):
        return

class log_logistic:
    def __init__(self, alpha, beta, time_col="efs_time"):
        self.alpha = alpha
        self.beta = beta
        self.time_col = time_col
    
    def predict(self, df):
        df = df.reset_index(drop=True).copy()
        
        # Log Logistic survival function
    
        y = 1 / (1 + (df[self.time_col] / (self.alpha)) ** (self.beta))

        return y.values

    def reset(self):
        return
        
def preprocess_data(x_train, x_valid, wb_list, beta, alpha=1, theta = 1):
    race_group = [0, 1, 2, 3, 4, 5]
    
    for race, wb in zip(race_group, wb_list):
      #  print(wb.predict(x_train[x_train["race_group"] == race].copy()))
        y_train_wb = wb.predict(x_train[x_train["race_group"] == race].copy())
        y_valid_wb = wb.predict(x_valid[x_valid["race_group"] == race].copy())
        
        x_train.loc[x_train["race_group"] == race, "y"] = y_train_wb
        x_valid.loc[x_valid["race_group"] == race, "y"] = y_valid_wb
        
    for dataset in [x_train, x_valid]:
        dataset.loc[dataset["efs"] == 0, "y"] = 1e-5
        dataset.loc[dataset["efs"] == 1, "y"] += 1e-5
    
    # Ensure beta is non-negative
    beta = abs(beta)

    y_train = alpha * x_train["y"] - beta * x_train["efs_time_norm"] + theta
    y_valid = alpha * x_valid["y"] - beta * x_valid["efs_time_norm"] + theta

    # Drop unnecessary columns
    drop_cols = ["y", "efs_time", "efs", "efs_time_norm"]
    x_train.drop(columns=drop_cols, inplace=True)
    x_valid.drop(columns=drop_cols, inplace=True)
    
    return x_train, y_train, x_valid, y_valid


In [ ]:
from xgboost import XGBRegressor, XGBClassifier
import xgboost as xgb
print("Using XGBoost version",xgb.__version__)

from catboost import CatBoostRegressor, CatBoostClassifier
import catboost as cb
print("Using CatBoost version",cb.__version__)

In [ ]:
def train_xgb(model_xgb, type, TRAIN = True, random_state=42, FOLDS = 10, aug = True):

    kf = KFold(n_splits=FOLDS, shuffle=True, random_state=random_state)
    np.random.seed(random_state)  
    random.seed(random_state)
    oof_xgb = np.zeros(len(train))
    pred_xgb = np.zeros(len(test))
    max_efs_time = train["efs_time"].max()
    train["efs_time_norm"] = train["efs_time"]/max_efs_time

    nan_columns = train.columns[train.isna().any()].tolist()

    if type == "weibull":
        scale_1 = np.random.normal(loc=8, scale= 0.2)
        shape_1 = np.random.normal(loc=1.8, scale= 0.2)
    
        scale_2 = np.random.normal(loc=5.8, scale= 0.2)
        shape_2 = np.random.normal(loc=3.2, scale= 0.2)
    
        scale_3 = np.random.normal(loc=7.2, scale= 0.2)
        shape_3 = np.random.normal(loc=2.8, scale= 0.2)
        
        scale_4 = np.random.normal(loc=8,   scale= 0.2)
        shape_4 = np.random.normal(loc=1.8, scale= 0.2)
    
        scale_5 = np.random.normal(loc=7.8, scale= 0.2)
        shape_5 = np.random.normal(loc=2.2, scale= 0.2)
    
        scale_6 = np.random.normal(loc=7.8, scale= 0.2)
        shape_6 = np.random.normal(loc=1.8, scale= 0.2)
    
    
        wb_list = [
            weibull(scale=scale_1, shape=shape_1),
            weibull(scale=scale_2, shape=shape_2),
            weibull(scale=scale_3, shape=shape_3),
            weibull(scale=scale_4, shape=shape_4),
            weibull(scale=scale_5, shape=shape_5),
            weibull(scale=scale_6, shape=shape_6),
        ]
        beta = 0.01
        
    if type == "log_logistic":
        alpha_list = np.random.normal(loc=[6, 6, 7, 6, 8, 8], scale=0.3, size=6)
        beta_list = np.random.normal(loc=[6, 10, 4, 10, 10, 10], scale=0.3, size=6)
        # Create the Weibull distributions
        wb_list = [log_logistic(alpha=s, beta=sh) for s, sh in zip(alpha_list, beta_list)]
        beta = 0.01
        
    if type == "exp":
        lambda_list = np.random.normal(loc=[0.20, 0.35, 0.2, 0.2, 0.15, 0.2], scale = 0.01, size = 6)
        wb_list = [exponential(lambda_=lamb) for lamb in lambda_list]
        beta = 0.01
        
    for i, (train_index, test_index) in enumerate(kf.split(train)):
        
        print("#"*25)
        print(f"### Fold {i+1}")
        print("#"*25)

        if aug:
            df_train_copy = train.loc[train_index,:].copy()
            df_train = train.loc[train_index,:].copy()
            df_train_aug = df_train_copy.copy()

            std_dev = 2
            random_samples = df_train_aug[(df_train_aug["efs"] == 1)].sample(n=5000, replace=False, random_state=random_state).copy()
               
            random_variance = np.random.normal(loc=random_samples["efs_time"], scale=std_dev, size=random_samples.shape[0])
            random_variance = np.maximum(random_variance, 1e-1)

            random_samples["efs_time"] = random_variance
            random_samples["efs_time_norm"] = random_samples["efs_time"]/max_efs_time
                
            df_train = pd.concat([df_train, random_samples], axis=0).reset_index(drop=True)

        else:
            df_train = train.loc[train_index,:].copy()

        x_train = df_train[FEATURES+["efs_time", "efs", "efs_time_norm"]].copy()
        
        x_valid = train.loc[test_index,FEATURES+["efs_time", "efs", "efs_time_norm"]].copy()
                    
        x_train, y_train, x_valid, y_valid = preprocess_data(x_train.copy(), x_valid.copy(), wb_list, beta=0.01, alpha=10, theta = 1)

        if TRAIN:   
            print("Size:", len(x_train))
            print("N_FEATURE: ", len(x_train.columns))
            
            model_xgb.fit(
                x_train, y_train,
                eval_set=[(x_valid, y_valid)],
                verbose=500
            )
            joblib.dump(model_xgb, f"model/xgboost/xgb_model_{type}_r{random_state}_f{i}.pkl")
        else:
            model_xgb = joblib.load(f"model/xgboost/xgb_model_{type}_r{random_state}_f{i}.pkl")
        
        oof_xgb[test_index] = model_xgb.predict(x_valid)

    pred_xgb = None
    y_true = train[["ID","efs","efs_time","race_group"]].copy()
    y_pred = train[["ID"]].copy()
    y_pred["prediction"] = oof_xgb
    m, cv_list = score(y_true.copy(), y_pred.copy(), "ID", PRINT = True)
    print(f"\nOverall CV for XGBoost =",m)
    
    np.save(f'model/xgb_cv/cv_{type}_r{random_state}.npy', cv_list)

    return oof_xgb, pred_xgb, m, cv_list

In [ ]:
def train_loop_xgb(type, random_states, TRAIN = True):
    model_xgb = XGBRegressor(
        objective="reg:tweedie",
        device="cuda",
        max_depth=6,
        colsample_bytree=0.2,
        subsample=0.8,
        n_estimators=50000,
        reg_lambda = 2,
        learning_rate=0.01,
        min_child_weight=160,
        enable_categorical = True,
        early_stopping_rounds=500,
        max_cat_to_onehot = 11,
    )
    
    # Initialize empty lists to store results
    oof_xgb = []
    pred_xgb = []
    cv_xgb =  []

    AUG = True
    for random_state in random_states:
        
        print("*" * 100)
        print("random state:", random_state)
        print("*" * 100)

        if random_state >= 20:
            AUG = False
            
        oof, pred, cv, cv_list = train_xgb(
            model_xgb,
            type = type,
            random_state=random_state,
            aug= True,
            TRAIN = TRAIN,
            FOLDS = 10
        )
            
        oof_xgb.append(oof)
        pred_xgb.append(pred)
        cv_xgb.append(cv_list)
    
        # Convert cv list to array for easier manipulation
        cv_xgb_array = np.array(cv_xgb)
        
        # Define weights for each race group
        weights_list = []
        for race_i in range(6):  # Assuming 5 race groups
            race_cv_scores = cv_xgb_array[:, race_i]
            weights_list.append(rankdata(race_cv_scores))  
        
        # Convert weights list to array
        weights_array = np.array(weights_list)
        
        # Normalize weights for each race
        weights_array /= np.sum(weights_array, axis=1, keepdims=True)
        
        # Prepare data for predictions
        y_true = train[["ID", "efs", "efs_time", "race_group"]].copy()
        y_pred = train[["ID"]].copy()
        
        # Calculate weighted ensemble predictions
        all_ranks = []
        
        for weights, oof in zip(weights_array.T, oof_xgb):  # Transposing to match dimensions
            weighted_oof = np.zeros_like(oof)
        
            for race_i in range(6):
                mask = train["race_group"] == race_i
                weighted_oof[mask] = weights[race_i] * (oof[mask] - oof[mask].mean()) / oof[mask].std()
            
            all_ranks.append(weighted_oof)
        
        # Convert to numpy array and take mean across all models
        all_ranks = np.array(all_ranks)
        y_pred["prediction"] = np.mean(all_ranks, axis=0)
        
        # Evaluate ensemble performance
        m, _ = score(y_true.copy(), y_pred.copy(), "ID", PRINT = True)
        print(f"\nOverall CV for Ensemble = {m}")

    return oof_xgb, pred_xgb, cv_xgb

In [ ]:
random_states = [r for r in range(0, 40)]

oof_xgb_wb, pred_xgb_wb, cv_xgb_wb = train_loop_xgb("weibull", random_states, TRAIN = True) 

# CatBoost
We train CatBoost model for 10 folds and achieve **CV 0.674**!

In [ ]:
import random

def train_catboost(random_state=42, FOLDS = 10, aug = True, TRAIN = True):

    kf = KFold(n_splits=FOLDS, shuffle=True, random_state=random_state)
    np.random.seed(random_state)  
    random.seed(random_state)
    oof_cat = np.zeros(len(train))
    pred_cat = np.zeros(len(test))
    max_efs_time = train["efs_time"].max()
    train["efs_time_norm"] = train["efs_time"]/max_efs_time
   
    race_group = [0, 1, 2, 3, 4, 5]

    nan_columns = train.columns[train.isna().any()].tolist()
    
    scale_1 = np.random.normal(loc=8, scale= 0.1)
    shape_1 = np.random.normal(loc=1.8, scale= 0.1)

    scale_2 = np.random.normal(loc=5.8, scale= 0.1)
    shape_2 = np.random.normal(loc=3.2, scale= 0.1)

    scale_3 = np.random.normal(loc=7.2, scale= 0.1)
    shape_3 = np.random.normal(loc=2.8, scale= 0.1)
    
    scale_4 = np.random.normal(loc=8,   scale= 0.1)
    shape_4 = np.random.normal(loc=1.8, scale= 0.1)

    scale_5 = np.random.normal(loc=7.8, scale= 0.1)
    shape_5 = np.random.normal(loc=2.2, scale= 0.1)

    scale_6 = np.random.normal(loc=7.8, scale= 0.1)
    shape_6 = np.random.normal(loc=1.8, scale= 0.1)
    
    wb_list = [
        weibull(scale=scale_1, shape=shape_1),
        weibull(scale=scale_2, shape=shape_2),
        weibull(scale=scale_3, shape=shape_3),
        weibull(scale=scale_4, shape=shape_4),
        weibull(scale=scale_5, shape=shape_5),
        weibull(scale=scale_6, shape=shape_6),
    ]

    for i, (train_index, test_index) in enumerate(kf.split(train)):
        if TRAIN:
            print("#"*25)
            print(f"### Fold {i+1}")
            print("#"*25)


        if aug:
            df_train_copy = train.loc[train_index,:].copy()
            df_train = train.loc[train_index,:].copy()
            df_train_aug = df_train_copy.copy()

            std_dev = 1.5
            random_samples = df_train_aug[(df_train_aug["efs"] == 1)].sample(n=5000, replace=False, random_state=random_state).copy()
               
              #  random_samples = df_train_aug.sample(n=int(len(df_train_aug)), replace=False, random_state=random_state)
            random_variance = np.random.normal(loc=random_samples["efs_time"], scale=std_dev, size=random_samples.shape[0])
            random_variance = np.maximum(random_variance, 1e-1)

            random_samples["efs_time"] = random_variance
            random_samples["efs_time_norm"] = random_samples["efs_time"]/max_efs_time                    

            df_train = pd.concat([df_train, random_samples], axis=0).reset_index(drop=True)
        else:
            df_train = train.loc[train_index,:].copy()

        x_train = df_train[FEATURES+["efs_time", "efs", "efs_time_norm"]].copy()
        x_valid = train.loc[test_index,FEATURES+["efs_time", "efs", "efs_time_norm"]].copy()

        x_train, y_train, x_valid, y_valid = preprocess_data(x_train.copy(), x_valid.copy(), wb_list, beta=0.01, alpha=0.5, theta = 0.1)

        if TRAIN:
            print("Size:", len(x_train))
            print("N_FEATURE: ", len(x_train.columns))

        # Create a CatBoost regressor with additional parameters
        if TRAIN:
            model_cat = CatBoostRegressor(
                task_type="GPU",
                bootstrap_type = "Bernoulli",
                grow_policy="Lossguide",
                learning_rate=0.01,
                iterations=50000,
                depth=8,
                one_hot_max_size=6,
                cat_features=CATS,
                early_stopping_rounds=250,
                loss_function="Tweedie:variance_power=1.5",
                min_data_in_leaf=60,
            )
    
            model_cat.fit(x_train,y_train,
                      eval_set=(x_valid, y_valid),
                      cat_features=CATS,
                      verbose=0)
            
            #model_cat.save_model("catboost_model.cbm")
            joblib.dump(model_cat, f"model/catboost/cat_model_r{random_state}_f{i}.pkl")
        else:
            model_cat = joblib.load(f"model/catboost/cat_model_r{random_state}_f{i}.pkl")
            
        # INFER OOF
        oof_cat[test_index] = model_cat.predict(x_valid)
        # INFER TEST
      #  pred_cat += model_cat.predict(x_test)

    if TRAIN:
        # COMPUTE AVERAGE TEST PREDS
        pred_cat = None
        y_true = train[["ID","efs","efs_time","race_group"]].copy()
        y_pred = train[["ID"]].copy()
        y_pred["prediction"] = oof_cat
        m, cv_list = score(y_true.copy(), y_pred.copy(), "ID", PRINT = True)
        print(f"\nOverall CV for CatBoost =",m)
    
        np.save(f'model/cat_cv/cv_r{random_state}.npy', cv_list)

    else:
        cv_list = np.load(f'model/cat_cv/cv_r{random_state}.npy', allow_pickle=True)
        m = np.mean(cv_list) - np.std(cv_list)
    return oof_cat, pred_cat, m, cv_list

In [ ]:
def train_loop_cat(random_states, TRAIN = True):
    # Initialize empty lists to store results
    oof_cat = []
    pred_cat = []
    cv_cat = []

    AUG = True
    for random_state in random_states:
        print("*" * 100)
        print("random state:", random_state)
        print("*" * 100)

        if random_state > 60:
            AUG = False
            
        oof, pred, cv, cv_list = train_catboost(
            random_state=random_state,
            aug=AUG,
            TRAIN = TRAIN
        )
        oof_cat.append(oof)
        pred_cat.append(pred)
        cv_cat.append(cv_list)
    
        # Convert cv list to array for easier manipulation
        cv_cat_array = np.array(cv_cat)
        
        # Define weights for each race group
        weights_list = []
        for race_i in range(6):  # Assuming 5 race groups
            race_cv_scores = cv_cat_array[:, race_i]
            weights_list.append(rankdata(race_cv_scores))  
        
        # Convert weights list to array
        weights_array = np.array(weights_list)
        
        # Normalize weights for each race
        weights_array /= np.sum(weights_array, axis=1, keepdims=True)
        
        # Prepare data for predictions
        y_true = train[["ID", "efs", "efs_time", "race_group"]].copy()
        y_pred = train[["ID"]].copy()
        
        # Calculate weighted ensemble predictions
        all_ranks = []
        for weights, oof in zip(weights_array.T, oof_cat):  # Transposing to match dimensions
            weighted_oof = np.zeros_like(oof)
        
            for race_i in range(6):
                mask = train["race_group"] == race_i
                weighted_oof[mask] = weights[race_i] * (oof[mask] - oof[mask].mean()) / oof[mask].std()
            
            all_ranks.append(weighted_oof)
        
        # Convert to numpy array and take mean across all models
        all_ranks = np.array(all_ranks)
        y_pred["prediction"] = np.mean(all_ranks, axis=0)
        
        # Evaluate ensemble performance
        m, cv_xgb = score(y_true.copy(), y_pred.copy(), "ID", PRINT = True)
        print(f"\nOverall CV for Ensemble = {m}")


    return oof_cat, pred_cat, cv_cat

In [ ]:
random_states = [r for r in range(40, 80)]

oof_cat, pred_cat, cv_cat = train_loop_cat(random_states, TRAIN=True) 